In [26]:
from Lab6 import *
from random import randint
from numpy import array,inf
from time import time
from threading import Thread
import threading
from math import sqrt,ceil
import warnings
warnings.simplefilter("ignore")

In [27]:
ev = threading.Event()
def stop():
    return ev.is_set()

### Naiwny test pierwszości (test pierwiastka kwadratowego)

In [30]:
def is_prime_naive_sqrt(n):
    if n <= 1:
        return False
    
    for i in range(2, ceil(sqrt(n))+1):
        if stop(): 
            return
        if n % i == 0:
            return False    
    return True     

### Test pierwszości Fermata

In [29]:
def is_prime_fermat(n, k):
    for i in range(k):
        if stop(): 
            return
        if n <= 3:
            return True
        else:
            t = randint(2,(n-2))
            if pow(t,n-1,n) != 1:
                return False
        return True

Miller-Rabin test:
- Input: n > 2, an odd integer to be tested for primality;
  -k, a parameter that determines the accuracy of the test
- Output: composite if n is composite, otherwise probably prime
  - write n − 1 as 2s·d with d odd by factoring powers of 2 from n − 1
  - LOOP: repeat k times:
    - pick a randomly in the range [2, n − 1]
    - x ← a d mod n
    - if x = 1 or x = n − 1 then do next LOOP
    - repeat s − 1 times:
      - x ← x 2 mod n
      - if x = 1 then return composite
      - if x = n − 1 then do next LOOP
    - return composite
  - return probably prime

In [28]:
def is_prime_miller_rabin(n, k):
    """
    Input: n > 2, an odd integer to be tested for primality;
       k, a parameter that determines the accuracy of the test
    Output: composite if n is composite, otherwise probably prime
    write n − 1 as 2s·d with d odd by factoring powers of 2 from n − 1
    LOOP: repeat k times:
       pick a randomly in the range [2, n − 1]
       x ← a d mod n
       if x = 1 or x = n − 1 then do next LOOP
       repeat s − 1 times:
          x ← x 2 mod n
          if x = 1 then return composite
          if x = n − 1 then do next LOOP
       return composite
    return probably prime
    """
    if not n&1 or n==1:
        return False
    s=1
    d=n-1
    while not d&1:
        s*=2
        d//=2
    for i in range(k):
        a = randint(2,n-1)
        x = pow(a,d,n)
        if x==1 or x==n-1:
            continue
        for j in range(s-1):
            if stop(): 
                return
            x = pow(x,2,n)
            if x==1:
                return False
            if x==n-1:
                break
        if x==n-1:
            continue
        return False
    return True

In [31]:
def compareExecutionTime(A, B, reps=1, timelimit=3, **kwargs):
    t1 = []
    t2 = []
    res=None
    r1=None
    r2=None
    def f1(**kwargs):
        nonlocal r1,t1
        r1=A(**kwargs)
        if not stop():
            t1.append(time() - time1)
    def f2(**kwargs):
        nonlocal r2,t2
        r2=B(**kwargs)
        if not stop():
            t2.append(time() - time2)
    for i in range(reps):
        r1,r2=None,None
        time1 = time()
        
        l=len(t1)
        
        p1 = Thread(target = f1, kwargs = kwargs)
        p1.start()
        p1.join(timeout = timelimit)
        while time()-time1<timelimit and p1.is_alive(): pass
        ev.set()
        while p1.is_alive(): pass
        ev.clear()
        
        if len(t1)==l:
            t1.append(inf)
        
        l=len(t2)
        time2 = time()
        p2 = Thread(target = f2, kwargs = kwargs)
        p2.start()
        p2.join(timeout = timelimit)
        while time()-time2<timelimit and p2.is_alive(): pass
        ev.set()
        while p2.is_alive(): pass
        ev.clear()
        
        if len(t2)==l:
            t2.append(inf)
        
        assert r1 is None or r2 is None or r1==r2
        if res is None:
            res=r1
        r1=None
        r2=None
    t11 = array(t1, dtype=float)
    t21 = array(t2, dtype=float)
    
    return [(t11.mean(), t11.std()), (t21.mean(), t21.std()), res]

Execution time analysis:
- Given two lists of prime and composite numbers
- For each number:
  - Check if a primality test function returns the expected value
  - Test the execution time
  - Print mean and standard deviation of the execution time for each number

In [35]:
prime=[
    3,
    17,
    101,
    1013,
    10099,
    103141,
    1016783,
    10103099,
    107707549,
    10770592813,
    107705911723,
    4547337172376300111955330758342147474062293202868155909489
]
composite=[
    2,
    3*17,
    556,
    800001,
    1007*101,
    103141*1016783,
    245709213523*5299871829441939,
    5059489012128901*1298582838728545,
    4547337172376300111955330758342147474062293202868155909393
]

algorithms=[
    is_prime_naive_sqrt,
    is_prime_fermat,
    is_prime_miller_rabin
]
args=[
    {"n":1},
    {"n":1,"k":1},
    {"n":1,"k":1}
]
    

def execFor(algo, _n, expected, **kwargs):
    A=algo
    kw=dict()
    print("n =",_n)
    kw['n']=_n
    k=min(10000,max(5,p//1000))
    if "k" in kwargs:
        print("k =",k)
        kw['k']=k
    
    time1,time2,res=compareExecutionTime(A,A,timelimit=1, reps=3,**kw)
    
    print("Mean exec. time of %s:        %.15f" % (A.__name__, time1[0]))
    print("Std. of the exec. time of %s: %.15f" % (A.__name__, time1[1]))
    print("Result (is prime):",res)
    print("\n\n")

for i in range(len(algorithms)):
    algo=algorithms[i]
    kw=args[i]
    print("\n\n\n")
    print(algo.__name__)
    print("\n\n")
    
    for p in prime:
        execFor(algo, p, True,**kw)
    
    for c in composite:
        execFor(algo, c, False,**kw)
    print(end="\n\n\n\n\n")





is_prime_naive_sqrt



n = 3
Mean exec. time of is_prime_naive_sqrt:        0.000072797139486
Std. of the exec. time of is_prime_naive_sqrt: 0.000102950701963
Result (is prime): True



n = 17
Mean exec. time of is_prime_naive_sqrt:        0.000472466150920
Std. of the exec. time of is_prime_naive_sqrt: 0.000620576718758
Result (is prime): True



n = 101
Mean exec. time of is_prime_naive_sqrt:        0.000000000000000
Std. of the exec. time of is_prime_naive_sqrt: 0.000000000000000
Result (is prime): True



n = 1013
Mean exec. time of is_prime_naive_sqrt:        0.000000000000000
Std. of the exec. time of is_prime_naive_sqrt: 0.000000000000000
Result (is prime): True



n = 10099
Mean exec. time of is_prime_naive_sqrt:        0.000000000000000
Std. of the exec. time of is_prime_naive_sqrt: 0.000000000000000
Result (is prime): True



n = 103141
Mean exec. time of is_prime_naive_sqrt:        0.000333468119303
Std. of the exec. time of is_prime_naive_sqrt: 0.000471595136938
Result 

Mean exec. time of is_prime_miller_rabin:        0.029217481613159
Std. of the exec. time of is_prime_miller_rabin: 0.007746362956345
Result (is prime): True



n = 10770592813
k = 10000
Mean exec. time of is_prime_miller_rabin:        0.051224470138550
Std. of the exec. time of is_prime_miller_rabin: 0.005072631972207
Result (is prime): True



n = 107705911723
k = 10000
Mean exec. time of is_prime_miller_rabin:        0.053974310557048
Std. of the exec. time of is_prime_miller_rabin: 0.006351865865977
Result (is prime): True



n = 4547337172376300111955330758342147474062293202868155909489
k = 10000
Mean exec. time of is_prime_miller_rabin:        0.599021275838216
Std. of the exec. time of is_prime_miller_rabin: 0.005035179729445
Result (is prime): True



n = 2
k = 10000
Mean exec. time of is_prime_miller_rabin:        0.000332911809285
Std. of the exec. time of is_prime_miller_rabin: 0.000470808395766
Result (is prime): False



n = 51
k = 10000
Mean exec. time of is_prime_miller_

#### $$\textbf{Conclusions}$$
1. Test Millera-Rabina jest probabilistycznym testem pierwszości,
    pozwala z dużym prawdopodobieństwem stwierdzić, czy dana liczba jest liczbą pierwszą.
    Jest znacznie szybszy w porównaniu z nieprobabilistycznym testem pierwszości pierwiastka kwadratowego,
    a dokładność wyników testu można określić i zmienić.
    Ten test wyróżnia się w analizie pierwszorzędności dużych liczb (np. >1e6),
    ze względu na skrócony czas wykonania i zużycie zasobów.
    Wadą jest to, że jego wyniki nie są jednoznaczne z prawdopodobieństwem 100%.
    Niemniej jednak test Millera-Rabina jest powszechnie znany i stosowany. To jest
    uważany za klasyczny test pierwszości.
2. Fermat test (conclusions)

3. Square root test (conclusions)
